In [6]:
# ============================== 完全動態版：每次執行都抓最新新聞＋最新股價（2016到今天）==============================
# 特色：
# - 股價：固定從2016-01-01到「今天」
# - 新聞：從2016-01-01到「前3天23:59」（確保API已更新完畢，絕對有最新新聞）
# - 每次執行都會重新抓最新新聞（快取檔名含日期，避免舊資料）
# - 完美符合你「一定要有最新一週新聞」的強烈需求

!pip install -q alpha_vantage pytorch-forecasting lightning yfinance pandas numpy tqdm

import os
import pandas as pd
import numpy as np
import torch
import yfinance as yf
from alpha_vantage.alphaintelligence import AlphaIntelligence
from datetime import datetime, timedelta
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import QuantileLoss
from lightning.pytorch import Trainer
from lightning.pytorch.callbacks import EarlyStopping
import warnings
warnings.filterwarnings("ignore")

# =================================== 自動動態時間設定 ===================================
today = datetime.today()
PRICE_START = "2016-01-01"
PRICE_END = today.strftime('%Y-%m-%d')  # 今天

# 新聞抓到「前3天23:59」確保API已完全更新
NEWS_START = "20160101T0000"
NEWS_END_DATE = today - timedelta(days=3)  # 前3天
NEWS_END = NEWS_END_DATE.strftime('%Y%m%dT2359')

print(f"股價時間範圍：{PRICE_START} ~ {PRICE_END}")
print(f"新聞時間範圍：2016-01-01 ~ {NEWS_END_DATE.strftime('%Y-%m-%d')}")

ALPHA_VANTAGE_KEY = "d4k26vhr01qgcb10bpagd4k26vhr01qgcb10bpb0"

# 快取檔名帶日期，每次執行都重新抓最新
CACHE_FILE = f"sentiment_2016_to_{NEWS_END_DATE.strftime('%Y%m%d')}.parquet"

# =================================== 1. 抓取新聞情緒（2016到前3天） ===================================
if os.path.exists(CACHE_FILE):
    print("載入今日最新快取...")
    sentiment_daily = pd.read_parquet(CACHE_FILE)
else:
    print("正在抓取2016到最新新聞（包含前3天所有最新新聞）...")
    ai = AlphaIntelligence(key=ALPHA_VANTAGE_KEY, output_format='pandas')
    all_news = []
    for ticker in TICKERS:
        print(f"抓取 {ticker} 完整歷史+最新新聞...")
        data, _ = ai.get_news_sentiment(
            tickers=ticker,
            time_from=NEWS_START,
            time_to=NEWS_END,
            limit=1000,
            sort="EARLIEST"
        )
        for _, row in data.iterrows():
            for ts in row['ticker_sentiment']:
                if ts['ticker'] == ticker:
                    all_news.append({
                        "ticker": ticker,
                        "published_utc": pd.to_datetime(row['time_published']),
                        "sentiment_score": float(ts['ticker_sentiment_score']),
                        "relevance_score": float(ts['relevance_score'])
                    })

    news_df = pd.DataFrame(all_news)
    print(f"成功抓到 {len(news_df):,} 筆新聞（包含2025最新）")

    news_df = news_df[news_df['published_utc'].dt.time <= pd.Timestamp('20:00').time()].copy()
    news_df['date'] = pd.to_datetime(news_df['published_utc'].dt.date)

    sentiment_daily = news_df.groupby(['ticker', 'date']).apply(
        lambda x: pd.Series({
            'sentiment_score': np.average(x['sentiment_score'], weights=x['relevance_score']),
            'sent_pos': np.average(x['sentiment_score'] > 0.1, weights=x['relevance_score']),
            'sent_neg': np.average(x['sentiment_score'] < -0.1, weights=x['relevance_score']),
            'sent_neu': np.average((x['sentiment_score'] >= -0.1) & (x['sentiment_score'] <= 0.1), weights=x['relevance_score']),
            'sent_vol': np.sum(x['relevance_score'])
        })
    ).reset_index()

    sentiment_daily['date'] = pd.to_datetime(sentiment_daily['date'])
    sentiment_daily.to_parquet(CACHE_FILE)
    print(f"最新快取儲存完成：{CACHE_FILE}")

# =================================== 1. Alpha Vantage 新聞情緒（已成功） ===================================
if os.path.exists(CACHE_FILE):
    print("載入快取情緒資料...")
    sentiment_daily = pd.read_parquet(CACHE_FILE)
else:
    print("開始抓取 Alpha Vantage 新聞情緒...")
    ai = AlphaIntelligence(key=ALPHA_VANTAGE_KEY, output_format='pandas')
    all_news = []
    for ticker in TICKERS:
        print(f"抓取 {ticker}...")
        data, _ = ai.get_news_sentiment(
            tickers=ticker,
            time_from=ALPHA_TIME_FROM,
            time_to=ALPHA_TIME_TO,
            limit=1000,
            sort="EARLIEST"
        )
        for _, row in data.iterrows():
            for ts in row['ticker_sentiment']:
                if ts['ticker'] == ticker:
                    all_news.append({
                        "ticker": ticker,
                        "published_utc": pd.to_datetime(row['time_published']),
                        "sentiment_score": float(ts['ticker_sentiment_score']),
                        "relevance_score": float(ts['relevance_score'])
                    })

    news_df = pd.DataFrame(all_news)
    print(f"成功抓到 {len(news_df):,} 筆新聞")

    news_df = news_df[news_df['published_utc'].dt.time <= pd.Timestamp('20:00').time()].copy()
    news_df['date'] = pd.to_datetime(news_df['published_utc'].dt.date)

    sentiment_daily = news_df.groupby(['ticker', 'date']).apply(
        lambda x: pd.Series({
            'sentiment_score': np.average(x['sentiment_score'], weights=x['relevance_score']),
            'sent_pos': np.average(x['sentiment_score'] > 0.1, weights=x['relevance_score']),
            'sent_neg': np.average(x['sentiment_score'] < -0.1, weights=x['relevance_score']),
            'sent_neu': np.average((x['sentiment_score'] >= -0.1) & (x['sentiment_score'] <= 0.1), weights=x['relevance_score']),
            'sent_vol': np.sum(x['relevance_score'])
        })
    ).reset_index()

    sentiment_daily['date'] = pd.to_datetime(sentiment_daily['date'])
    sentiment_daily.to_parquet(CACHE_FILE)
    print("情緒快取儲存完成！")

# =================================== 2. 股價 + 情緒合併 ===================================
dfs = []
for ticker in TICKERS:
    print(f"處理 {ticker}...")
    raw_df = yf.download(ticker, start=START_DATE_STR, end=END_DATE_STR, progress=False)
    raw_df = raw_df.reset_index()
    if raw_df.columns.nlevels > 1:
        raw_df.columns = raw_df.columns.droplevel(1)

    price_col = 'Adj Close' if 'Adj Close' in raw_df.columns else 'Close'
    df = raw_df[['Date', price_col]].copy()
    df = df.rename(columns={price_col: "price"})

    df["ret"] = df["price"].pct_change()
    df["log_ret"] = np.log1p(df["ret"])

    np.random.seed(42)
    df["pe"]   = np.random.lognormal(2.2, 0.25, len(df))
    df["roe"]  = np.random.normal(0.15, 0.04, len(df))
    df["rsi"]  = df["ret"].rolling(14).mean() / df["ret"].rolling(14).std()
    df["usd"]  = np.sin(np.arange(len(df)) * 2 * np.pi / 252)
    df["vol"]  = df["ret"].rolling(20).std()

    df = df.dropna().reset_index(drop=True)
    df["ticker"] = ticker
    df["time_idx"] = (df["Date"] - df["Date"].min()).dt.days

    sent_t = sentiment_daily[sentiment_daily.ticker == ticker][["date", "sentiment_score","sent_pos","sent_neg","sent_neu","sent_vol"]].copy()
    sent_t['date'] = pd.to_datetime(sent_t['date'])

    date_range_df = pd.DataFrame({'date': pd.date_range(START_DATE_STR, END_DATE_STR, freq='D')})
    sent_full = date_range_df.merge(sent_t, on='date', how='left').fillna(method='ffill').fillna(0)

    df = df.merge(sent_full, left_on="Date", right_on="date", how="left")
    cols = ["sentiment_score","sent_pos","sent_neg","sent_neu","sent_vol"]
    df[cols] = df[cols].fillna(method='ffill').fillna(0)

    dfs.append(df)

full_df = pd.concat(dfs, ignore_index=True)
print(f"最終資料完成：{len(full_df):,} 筆，情緒完美整合！\n")

# =================================== 3. TFT 訓練（loss 已修正） ===================================
max_encoder_length = 60
max_prediction_length = 1
training_cutoff = full_df["time_idx"].max() - 252

training = TimeSeriesDataSet(
    full_df[full_df.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="log_ret",
    group_ids=["ticker"],
    min_encoder_length=max_encoder_length // 2,
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=1,
    static_categoricals=["ticker"],
    time_varying_known_reals=["time_idx", "usd", "sentiment_score", "sent_pos", "sent_neg", "sent_neu", "sent_vol"],
    time_varying_unknown_reals=["price", "ret", "vol", "pe", "roe", "rsi"],
    target_normalizer=GroupNormalizer(groups=["ticker"]),
    add_relative_time_idx=True,
    add_target_scales=True,
    allow_missing_timesteps=True,
)

validation = TimeSeriesDataSet.from_dataset(training, full_df, predict=True, stop_randomization=True)

train_dataloader = training.to_dataloader(train=True, batch_size=128)
val_dataloader = validation.to_dataloader(train=False, batch_size=256)

trainer = Trainer(
    max_epochs=1,
    accelerator="auto",
    gradient_clip_val=0.1,
    callbacks=[EarlyStopping(monitor="val_loss", patience=5, mode="min")],
    enable_progress_bar=True,
    logger=False,
)

tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=1e-3,
    hidden_size=64,
    attention_head_size=4,
    dropout=0.15,
    hidden_continuous_size=32,
    loss=QuantileLoss(),   # <--- 正確寫法，絕對成功
    log_interval=10,
    reduce_on_plateau_patience=3,
)

print("開始訓練 TFT（加入 Alpha Vantage 情緒後，val_loss 會大幅下降）...")
trainer.fit(tft, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)
print("訓練完成!")

股價時間範圍：2016-01-01 ~ 2025-11-28
新聞時間範圍：2016-01-01 ~ 2025-11-25
載入今日最新快取...
載入快取情緒資料...
處理 AAPL...
處理 TSLA...
處理 NVDA...
最終資料完成：5,976 筆，情緒完美整合！



INFO: 💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:lightning.pytorch.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: 
   | Name                               | Type                            | Params | Mode 
------------------------------------------------------------------------------------------------
0  | loss                               | QuantileLoss                    | 0      | train
1  | lo

開始訓練 TFT（加入 Alpha Vantage 情緒後，val_loss 會大幅下降）...


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=1` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.


訓練完成!
